In [1]:
cd ../src

/storage2/home2/e1-313-15477/govind/s2slp/src


In [15]:
import data_preparer
import link_predictor

In [14]:
data_preparer.main()

100%|██████████| 10/10 [00:02<00:00,  3.66it/s]


In [16]:
link_predictor.main()

100%|██████████| 10/10 [00:14<00:00,  1.43s/it]

                  train            test
cn        0.5274+-0.003  0.5254+-0.0051
cn_      0.5272+-0.0031  0.5255+-0.0054
cn_mean  0.5273+-0.0031  0.5255+-0.0052
